In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a href="https://colab.research.google.com/github/google-research/language-table/blob/main/language_table/examples/language_table_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Run In Google Colab"/></a>
  </td>
</table>


# Install Dependencies


In [ ]:
# !pip install git+https://github.com/google-research/language-table.git
!pip install mediapy
!pip install matplotlib==3.1.3

# Import Modules


In [ ]:
import os

from language_table.environments import blocks
from language_table.environments import language_table
from language_table.environments.rewards import block2block

from matplotlib import pyplot as plt
import mediapy
import numpy as np

import tensorflow_datasets as tfds

# Language Table Environment

The Language Table sim environment contains many different configurations for different tasks.

## Block Modes

- BLOCK_8: 8 blocks on the board: red moon, red pentagon, blue moon, blue cube, green cube, green star, yellow star, and yellow pentagon.
- BLOCK_4: 4 blocks on the board: red moon, blue cube, green star, and yellow pentagon.
- N_CHOOSE_K: Number of blocks on the board varies (total of 12 possible blocks).

## Rewards

| Reward                             | Description                                                |
| ---------------------------------- | ---------------------------------------------------------- |
| BlockToBlockReward                 | Move a block to another block.                             |
| BlockToAbsoluteLocationReward      | Move a block to a specific absolute location on the board. |
| BlockToRelativeLocationReward      | Move a block to a direction relative to itself.            |
| BlockToBlockRelativeLocationReward | Move a block to the particular side of another block.      |
| SeparateBlocksReward               | Separate 2 blocks.                                         |
| PlayReward                         | Freeform play, no particular goal.                         |


In [ ]:
def decode_inst(inst):
    """Utility to decode encoded language instruction"""
    return bytes(inst[np.where(inst != 0)].tolist()).decode("utf-8")


# Create an example with the BlockToBlockReward
reward_factory = block2block.BlockToBlockReward  # CHANGEME: change to another reward.

env = language_table.LanguageTable(
    block_mode=blocks.LanguageTableBlockVariants.BLOCK_8,
    reward_factory=reward_factory,
    seed=0,
)
obs = env.reset()

Show the RGB image and print the instruction.


In [ ]:
plt.imshow(obs["rgb"])
print(decode_inst(obs["instruction"]))

Calling env.render() also gives you the RGB image with the text rendered above. To turn off rendering the text, pass `render_text_in_image=False` in the environment constructor.


In [ ]:
plt.imshow(env.render())

Simple example of stepping the environment.


In [ ]:
for _ in range(3):
    env.reset()
    fig, axes = plt.subplots(1, 5, figsize=(15, 5))
    for ax in axes:
        obs, _, _, _ = env.step(np.array([0.1, 0]))
        ax.set_axis_off()
        ax.imshow(obs["rgb"])
    plt.show()

# Language Table Datasets

See [full details]((https://github.com/google-research/language-table#datasets) on the variants of Language Table Datasets here.

Here is a quick overview:

| Dataset                                        | Real/sim | Controlled by | Language-labeled by | # episodes |
| ---------------------------------------------- | -------- | ------------- | ------------------- | ---------: |
| language_table                                 | real     | human         | human               |    442,226 |
| language_table_sim                             | sim      | human         | human               |    181,020 |
| language_table_blocktoblock_sim                | sim      | human         | scripted            |      8,000 |
| language_table_blocktoblock_4block_sim         | sim      | human         | scripted            |      8,298 |
| language_table_blocktoblock_oracle_sim         | sim      | oracle        | scripted            |    200,000 |
| language_table_blocktoblockrelative_oracle_sim | sim      | oracle        | scripted            |    200,000 |
| language_table_blocktoabsolute_oracle_sim      | sim      | oracle        | scripted            |    200,000 |
| language_table_blocktorelative_oracle_sim      | sim      | oracle        | scripted            |    200,000 |
| language_table_separate_oracle_sim             | sim      | oracle        | scripted            |    200,000 |

See the [RLDS Tutorial](https://colab.research.google.com/github/google-research/rlds/blob/main/rlds/examples/rlds_tutorial.ipynb) for more details on the RLDS data format.


In [ ]:
DATASET_VERSION = "0.0.1"
DATASET_NAME = "language_table"  # CHANGEME: change this to load another dataset.

dataset_directories = {
    "language_table": "gs://gresearch/robotics/language_table",
    "language_table_sim": "gs://gresearch/robotics/language_table_sim",
    "language_table_blocktoblock_sim": "gs://gresearch/robotics/language_table_blocktoblock_sim",
    "language_table_blocktoblock_4block_sim": "gs://gresearch/robotics/language_table_blocktoblock_4block_sim",
    "language_table_blocktoblock_oracle_sim": "gs://gresearch/robotics/language_table_blocktoblock_oracle_sim",
    "language_table_blocktoblockrelative_oracle_sim": "gs://gresearch/robotics/language_table_blocktoblockrelative_oracle_sim",
    "language_table_blocktoabsolute_oracle_sim": "gs://gresearch/robotics/language_table_blocktoabsolute_oracle_sim",
    "language_table_blocktorelative_oracle_sim": "gs://gresearch/robotics/language_table_blocktorelative_oracle_sim",
    "language_table_separate_oracle_sim": "gs://gresearch/robotics/language_table_separate_oracle_sim",
}

In [ ]:
dataset_path = os.path.join(dataset_directories[DATASET_NAME], DATASET_VERSION)
builder = tfds.builder_from_directory(dataset_path)
episode_ds = builder.as_dataset(split="train")

print(episode_ds.element_spec)

## Loading Episodes

Take the first episode in the dataset, and make a video from the frames


In [11]:
episode = next(iter(episode_ds.take(1)))

frames = []
for step in episode["steps"].as_numpy_iterator():
    frames.append(step["observation"]["rgb"])

mediapy.show_video(frames, title=decode_inst(step["observation"]["instruction"]), fps=5)

2024-05-10 13:41:02.855866: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


RuntimeError: Program 'ffmpeg' is not found; perhaps install ffmpeg using 'apt install ffmpeg'.

## Loading Steps

Take the first 100 steps of the dataset, and make a combined video of the frames


In [ ]:
steps_ds = episode_ds.flat_map(
    lambda x: x["steps"]
)  # get the dataset as individual steps

# Make the first 100 steps of the dataset into a video.
frames = []
for step in steps_ds.take(100).as_numpy_iterator():
    frames.append(step["observation"]["rgb"])

mediapy.show_video(frames, fps=5)